In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas

*** Importing Data ***

In [ ]:
df_master=pd.read_csv(r'C:\Users\Kumar149348\Downloads\loan.csv')

### Checking Data Quality 

In [ ]:
df_master.info()

In [ ]:
# 39717 observations and 111 columns which matches with the data dictionary. So Imported data shape is correct

In [ ]:
df_master.head()

In [ ]:
# Setting the display option to show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_master.dtypes

#### Insights 

No issues in importing the data. Data Types also seeems to be correct.


### Understanding Data

In [ ]:
df_master.describe()

In [ ]:
df_master['loan_status'].value_counts().hvplot.bar(
    title="Loan Status Counts", xlabel='Loan Status', ylabel='Count', 
    width=500, height=350
)

### Target variable "Loan Status" is categorical and it is highly skewed in the data

In [ ]:
# identifying null columns---why remove them---coz they won't have any role to play in driving the default

df_master.isnull().sum()

In [ ]:
fill_rate=df_master.isna().mean().sort_values(ascending=False).round(4)*100

In [ ]:
# fill rate percentages for each columns

fill_rate

### Columns that need to be dropped and why

1. all columns with 0 fill rate-- as they would have no role in driving the defaults
2. row identifiers like 1d and member id-- as they would have no role in driving the defaults
3. text columns like url and desc-- as they would have no role in driving the defaults

In [ ]:
df_master.dropna(axis=1,how='all',inplace=True) 

In [ ]:
df_master=df_master.drop(['id','member_id','desc', 'url'],axis=1)

In [ ]:
df_master.info()

## Missing value data truncation 2nd iteration

In [ ]:
# Find columns with missing values
cols_with_missing = df_master.columns[df_master.isnull().any()].tolist()

df_missingValues = df_master[cols_with_missing]

sns.heatmap(df_missingValues.isnull(), cbar=False, cmap = 'Blues')
plt.show()

## missing value column - emp_title

In [ ]:
print(df_master['emp_title'].value_counts())

# Drop column 'title'
loan_data = df_master.drop(columns=['emp_title'])

#### Observation

From the analysis of the emp_title, unless we have ranking for the titles, or grades for them it would not serve any purpose in analysis.

So we decide to drop emp_title column. 

In [ ]:
## missing value column - emp_length

In [ ]:

print(df_master['emp_length'].value_counts())

print(df_master['emp_length'].unique())

In [ ]:
# Analysis of missing value column - emp_length
# "Bivariate analysis" to take a decision between the "emp_length" and the target variable


# Create a crosstab dataframe
cross_df = pd.crosstab(df_master['emp_length'], df_master['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)
cross_df

In [ ]:
# Bivariate Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by Employment length')
plt.show()


### Insights

all the categories of employment length are showing consistent percentage of people in defaults

Hence we would compute the missing value, and replace it with a complete constant - "Unknown" 


In [ ]:
df_master['emp_length'].fillna('Unknown', inplace=True)

In [ ]:
#### Analysis of missing value column - title

print(df_master['title'].value_counts())


### Insights


From the analysis of the title, unless we have ranking for the titles, or grades for them it would not serve any purpose in analysis.

So we decide to drop title column

In [ ]:
# Drop column 'title'
df_master = df_master.drop(columns=['title'])

#### Analysis of missing value column - mths_since_last_delinq
Since this is a column which is recorded, we cannot impute the missing value. We let it remain as it is. 



In [ ]:
df_master['mths_since_last_delinq'].fillna('Unknown', inplace=True)

#### Analysis of missing value column - mths_since_last_record
Since this is a column which is recorded, we cannot impute the missing value. We let it remain as it is. 



In [ ]:
df_master['mths_since_last_record'].fillna('Unknown', inplace=True)

#### Analysis of missing value column - revol_util

"Revolving utilization", also known as "credit utilization ratio", is a financial term commonly used in the credit industry, particularly with revolving lines of credit such as credit cards and personal lines of credit.

This cannot be calculated or assumed based on the available data as it is a "standard formula". 

We can either assume 0% utilization or we can drop of the rows as it is not explicity stated. 

We would drop all rows where reov_util is not available for our analysis.

In [ ]:
df_master = df_master.dropna(subset=['revol_util'])


#### Analysis of missing value column - last_pymnt_d
Since this is a column which is recorded, we cannot impute the missing value. However we have last_pymnt_amnt which seems to be related to last_pymnt_d. 

The date not getting recorded seems to be a human error. With the last_pymnt_amnt field having all values, it can be used for any feature creation. 

So we would drop the column last_pymnt_d

In [ ]:
# Drop column 'last_pymnt_d'
df_master = df_master.drop(columns=["last_pymnt_d"])

#### Analysis of missing value column - next_pymnt_d
Since this is a column which is recorded, we cannot impute the missing value. We would add a constant - "Unknown". 




In [ ]:
df_master['next_pymnt_d'].fillna('Unknown', inplace=True)

### Columns with very less missing data
Bivariate is used to establish the logic for any imputation

In [ ]:
#### Analysis of missing value - last_credit_pull_d

null_rows = df_master[df_master['last_credit_pull_d'].isnull()][['loan_status', 'last_credit_pull_d']]
null_rows['last_credit_pull_d'].fillna('Unknown', inplace=True)

In [ ]:
# Create a crosstab dataframe
cross_df = pd.crosstab(null_rows['last_credit_pull_d'], null_rows['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by last_credit_pull')
plt.show()

Even if it a fraction of missing data, it leads us to believe that all people who do not went for credit pull, 
have been charged off, we can assume people not using this facility are not sure of making full payment 
or possibly given up on business. We would as of now assume a constant - Unknown for this date field.

In [ ]:
df_master['last_credit_pull_d'].fillna('Unknown', inplace=True)

In [ ]:
#### Analysis for missing value column - collections_12_mths_ex_med

# Create a crosstab dataframe
cross_df = pd.crosstab(df_master['collections_12_mths_ex_med'], df_master['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by collections_12_mths_ex_med')
plt.show()



As "collections_12_mths_ex_med" has a single value "0", it seems to be a constant, we can replace the missing values with mode - "0". 

In [ ]:
#### Analysis for missing value column - chargeoff_within_12_mths

# Create a crosstab dataframe
cross_df = pd.crosstab(df_master['chargeoff_within_12_mths'], df_master['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by chargeoff_within_12_mths')
plt.show()



As "chargeoff_within_12_mths" has a single value "0", it seems to be a constant, we can replace the missing values with mode - "0". 


In [ ]:

#### Analysis for missing value column - pub_rec_bankruptcies

#pub_rec_bankruptcies

print(df_master['pub_rec_bankruptcies'].value_counts())

# Create a crosstab dataframe
cross_df = pd.crosstab(df_master['pub_rec_bankruptcies'], df_master['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by Bankruptcies')
plt.show()

#### Insights

There seems to link between Recorded Bankruptcies and increase in cases of Charged Off loan status. 
There is also lot of Current accounts where recorded bankruptcies are 2.0. 
This is a public record, and we cannot assume whethere there was a bankruptcies or not for certain accounts. 

dropping rows where pub_rec_bankruptcies are not available. 


In [ ]:

# Select all rows where Column1 is null
#null_rows = loan_data[loan_data['pub_rec_bankruptcies'].isnull()]
null_rows = df_master[df_master['pub_rec_bankruptcies'].isnull()][['loan_status', 'pub_rec_bankruptcies']]
null_rows

null_rows['pub_rec_bankruptcies'].fillna('Unknown', inplace=True)

In [ ]:

# Create a crosstab dataframe
cross_df = pd.crosstab(null_rows['pub_rec_bankruptcies'], null_rows['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by Bankruptcies')
plt.show()

As the percentages of "Charged Off" are less here, we can replace the values with "0" or a "constant - Unknown' and use one-hot encoding (dummy variables) 

However here we would drop all rows where this particular column has null values, assuming "pub_rec_bankruptcies" is an important variable.


In [ ]:

## Drop rows where Column1 is null
## This operation returns a new DataFrame, so we assign it back to df to update it.

df_master = df_master.dropna(subset=['pub_rec_bankruptcies'])

In [ ]:

### Analysis of missing value column - tax_liens
                           

# Create a crosstab dataframe
cross_df = pd.crosstab(df_master['tax_liens'], df_master['loan_status'])

# Convert counts to percentage
cross_df = cross_df.div(cross_df.sum(axis=1), axis=0)

# Plot
cross_df.plot(kind='bar', stacked=True)
plt.ylabel('Fraction of Loans')
plt.title('Loan Status by tax_liens')
plt.show()



As "tax_liens" has a single value "0", it seems to be a constant, we can replace the missing values with mode - "0". 



### Univariate Analysis

In [ ]:
df_master.hist(bins=50, figsize=(20,15))
plt.show()

From the graphs column - "collections_12_mths_ex_med", "policy_code", "acc_now_delinq", "chargeoff_within_12_mths", "delinq_amt", "tax_liens" are all constant, and hence their analysis would not make any sense.

Hence dropping those columns.

In [ ]:
# Dropping column that are constant
df_master = df_master.drop(columns=["collections_12_mths_ex_med",
                                    "policy_code", 
                                    "acc_now_delinq",
                                    "chargeoff_within_12_mths",
                                    "delinq_amnt", 
                                    "tax_liens"])

In [ ]:
df_master.info()

In [ ]:
behaviour_var =  [
  "delinq_2yrs",
  "earliest_cr_line",
  "inq_last_6mths",
  "open_acc",
  "pub_rec",
  "revol_bal",
  "revol_util",
  "total_acc",
  "out_prncp",
  "out_prncp_inv",
  "total_pymnt",
  "total_pymnt_inv",
  "total_rec_prncp",
  "total_rec_int",
  "total_rec_late_fee",
  "recoveries",
  "collection_recovery_fee",
  "last_pymnt_amnt",
  "last_credit_pull_d",
  "application_type"]
behaviour_var

In [ ]:
#remove the behaviour variables from analysis as they won't decide the drivers for default as we want to identify 
#risky applicants and servicing records won't help

df_master = df_master.drop(behaviour_var, axis=1)
df_master.info()

In [ ]:
installment = df_master.hvplot.hist(
    y='installment', by='loan_status', subplots=False, 
    width=350, height=400, bins=50, alpha=0.4, 
    title="Installment by Loan Status", 
    xlabel='Installment', ylabel='Counts', legend='top'
)

loan_amnt = df_master.hvplot.hist(
    y='loan_amnt', by='loan_status', subplots=False, 
    width=350, height=400, bins=30, alpha=0.4, 
    title="Loan Amount by Loan Status", 
    xlabel='Loan Amount', ylabel='Counts', legend='top'
)

installment + loan_amnt

### Though installment seems to add noise but we shall bin it and see later

In [ ]:
# dropping zip_code and addr_state as they won't be the drivers for default
df_master = df_master.drop(['zip_code', 'addr_state'], axis=1)

In [ ]:
# filtering only fully paid or charged-off
df_master = df_master[df_master['loan_status'] != 'Current']
df_master['loan_status'] = df_master['loan_status'].apply(lambda x: 0 if x=='Fully Paid' else 1)

# converting loan_status to integer type
df_master['loan_status'] = df_master['loan_status'].apply(lambda x: pd.to_numeric(x))

# summarising the values
df_master['loan_status'].value_counts()

In [ ]:
def plot_cat(cat_var):
    sns.barplot(x=cat_var, y='loan_status', data=df_master)
    plt.show()
    

In [ ]:
plot_cat('grade')


Clearly, as the grade of loan goes from A to G, the default rate increases. This is expected because the grade is decided by Lending Club based on the riskiness of the loan. 

In [ ]:
# sub-grade: as expected - A1 is better than A2 better than A3 and so on 
plt.figure(figsize=(16, 6))
plot_cat('sub_grade')

In [ ]:
# home ownership: not a great discriminator
plot_cat('home_ownership')

In [ ]:
# verification_status: surprisingly, verified loans default more than not verifiedb
plot_cat('verification_status')

In [ ]:
# purpose: small business loans defualt the most, then renewable energy and education
plt.figure(figsize=(16, 6))
plot_cat('purpose')

In [1]:
from datetime import datetime
df_master['issue_d'] = df_master['issue_d'].apply(lambda x: datetime.strptime(x, '%b-%y'))


# extracting month and year from issue_date
df_master['month'] = df_master['issue_d'].apply(lambda x: x.month)
df_master['year'] = df_master['issue_d'].apply(lambda x: x.year)




# number of loans granted across years
df_master.groupby('year').year.count()

NameError: name 'df_master' is not defined

In [ ]:
# the default rate had suddenly increased in 2011, inspite of reducing from 2008 till 2010
plot_cat('year')

In [ ]:
# comparing default rates across months: not much variation across months
plt.figure(figsize=(16, 6))
plot_cat('month')

### For Continuous Variables

In [ ]:
# loan amount: the median loan amount is around 10,000
sns.distplot(df_master['loan_amnt'])
plt.show()

### binning the loan amount variable into small, medium, high, very high.

In [ ]:
# binning loan amount
def loan_amount(n):
    if n < 5000:
        return 'low'
    elif n >=5000 and n < 15000:
        return 'medium'
    elif n >= 15000 and n < 25000:
        return 'high'
    else:
        return 'very high'
        
df_master['loan_amnt'] = df_master['loan_amnt'].apply(lambda x: loan_amount(x))


In [ ]:
df_master['loan_amnt'].value_counts()

In [ ]:
# higher the loan amount, higher the default rate
plot_cat('loan_amnt')

In [ ]:
# convert funded amount invested to bins
df_master['funded_amnt_inv'] = df_master['funded_amnt_inv'].apply(lambda x: loan_amount(x))

In [ ]:
# funded amount invested
plot_cat('funded_amnt_inv')

In [ ]:
# convert interest rate to low, medium, high

#column int_rate is character type, convert it to float

df_master['int_rate'] = df_master['int_rate'].apply(lambda x: pd.to_numeric(x.split("%")[0]))

def int_rate(n):
    if n <= 10:
        return 'low'
    elif n > 10 and n <=15:
        return 'medium'
    else:
        return 'high'
    
    
df_master['int_rate'] = df_master['int_rate'].apply(lambda x: int_rate(x))

In [ ]:
# high interest rates default more, as expected
plot_cat('int_rate')

In [ ]:
#debt to income ratio

def dti(n):
    if n <= 10:
        return 'low'
    elif n > 10 and n <=20:
        return 'medium'
    else:
        return 'high'
    

df_master['dti'] = df_master['dti'].apply(lambda x: dti(x))

In [ ]:
#high dti translates into higher default rates, as expected
plot_cat('dti')

In [ ]:
# funded amount
def funded_amount(n):
    if n <= 5000:
        return 'low'
    elif n > 5000 and n <=15000:
        return 'medium'
    else:
        return 'high'
    
df_master['funded_amnt'] = df_master['funded_amnt'].apply(lambda x: funded_amount(x))

In [ ]:
plot_cat('funded_amnt')

In [ ]:
# installment
def installment(n):
    if n <= 200:
        return 'low'
    elif n > 200 and n <=400:
        return 'medium'
    elif n > 400 and n <=600:
        return 'high'
    else:
        return 'very high'
    
df_master['installment'] = df_master['installment'].apply(lambda x: installment(x))

In [ ]:
# the higher the installment amount, the higher the default rate but is actually a noise 
plot_cat('installment')

In [ ]:
# annual income
def annual_income(n):
    if n <= 50000:
        return 'low'
    elif n > 50000 and n <=100000:
        return 'medium'
    elif n > 100000 and n <=150000:
        return 'high'
    else:
        return 'very high'

df_master['annual_inc'] = df_master['annual_inc'].apply(lambda x: annual_income(x))

In [ ]:
# lower the annual income, higher the default rate
plot_cat('annual_inc')

In [ ]:
# emp_length and default rate
# not much of a predictor of default
plot_cat('emp_length')


In [ ]:
# paymnt_plan and default rate
# not much of a predictor of default
plot_cat('pymnt_plan')

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_master.corr(), annot=True, cmap='viridis')

In [ ]:
df_master.corr()['loan_status'].drop('loan_status').sort_values().hvplot.barh(
    width=600, height=400, 
    title="Correlation between Loan status and Numeric Features", 
    ylabel='Correlation', xlabel='Numerical Features', 
)

### Final Conclusion-:

Important features to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default are

1	funded_amnt
2	funded_amnt_inv
3	term
4	int_rate
5	grade
6	sub_grade
7	annual_inc
8	verification_status
9	issue_d 
10	purpose
11	dti
12	pub_rec_bankruptcies

Note-: issue_d as such is not a significant var but year var created out of it is significant
